In [207]:
from uuid import uuid4

import pandas as pd
from slugify import slugify
# go to https://ford-parts.com.ru/bitrix/admin/iblock_data_export.php?lang=ru
# Каталог -> Каталог товаров и Торговые предложения
# Вытащить категории, сабкатегории и скрестить с новой версией в Postgres
#

In [208]:
catalogue = pd.read_csv("data/category.csv")

In [209]:
catalogue = catalogue[(catalogue["IE_NAME"] != "1")
                      & (catalogue["IE_NAME"] != "0")
                      & (catalogue["IE_NAME"] != "01")
                      & (catalogue["IC_GROUP1"].notna())
                      & (catalogue["IC_GROUP2"].notna())]

In [210]:
catalogue = catalogue[
    [
        "IE_XML_ID",
        "IE_NAME",
        "IE_PREVIEW_PICTURE",
        "IE_PREVIEW_TEXT",
        "IC_GROUP1",
        "IC_GROUP2",
    ]
].rename(
    columns={
        "IE_XML_ID": "bitrix_id",
        "IE_NAME": "name",
        "IE_PREVIEW_PICTURE": "old_image",
        "IE_PREVIEW_TEXT": "cross_number",
        "IC_GROUP1": "category",
        "IC_GROUP2": "sub_category",
    }
)
ids = [str(uuid4()) for i in range(len(catalogue))]
catalogue['id'] = ids
catalogue["slug"] = [slugify(n) for n in catalogue["name"].str.lower()]

In [211]:
catalogue

,bitrix_id,name,old_image,cross_number,category,sub_category,id,slug
1012,F8-2,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/965/h6v1v60gi1ryckdh0tc7ay6ltnz...,"1930055, 1743698, 8M51R442A66EA",Кузов,Механизмы замков,5df32069-0a4e-43c1-a1ed-013f3072e97c,focus-ii-iii-c-max-kuga-mondeo-s-max-07-zamok-...
1013,F8-3,Focus II-III / C-Max / Kuga / Mondeo / S-Max 0...,/upload/iblock/da3/klvvib5jemjl1rp5rfraxiri73a...,"1859161, 8M51R442A66AC, 8M51 R442A66 DB, 15704...",Кузов,Механизмы замков,8b1cbb81-f54f-4248-96b2-19cbc025775e,focus-ii-iii-c-max-kuga-mondeo-s-max-07-zamok-...
1014,F8-6,Механизм замка Connect задней (высокой) двери ...,NaN,"1419513, 2T14 V43287 BP",Кузов,Механизмы замков,89aee4ea-f966-4e2d-9bef-83482f6d73ce,mekhanizm-zamka-connect-zadnei-vysokoi-dveri-v...
1015,F8-7,Механизм замка L ЛЕВОЙ выдвижной длинной двер...,/upload/iblock/3bc/4b6qcru899yl23shyk2ndyfvic7...,"1766561, YC15-V264A33-BK",Кузов,Механизмы замков,5bed408e-88d0-4bb6-88aa-ba20525f96a7,mekhanizm-zamka-l-levoi-vydvizhnoi-dlinnoi-dve...
1016,F8-8,Механизм замка R правой выдвижной длинной двер...,/upload/iblock/f18/n4qjnqyb01i5awupzy87xbd8w15...,YC15 B264A00 BL N - FSE 11-321-050 - 1C15 B264...,Кузов,Механизмы замков,7c3d5027-e1a8-498a-90f6-e7459299f30a,mekhanizm-zamka-r-pravoi-vydvizhnoi-dlinnoi-dv...
...,...,...,...,...,...,...,...,...
9650,F22-467,Датчик давления выхлопных газов Ford Transit 2...,NaN,"1600077, 6G9N-5L200-AB",Система топливная,Датчик давления воздуха (MAP / наддув / выхлоп),80a83218-7074-4e59-bd69-2d5daa8beb83,datchik-davleniia-vykhlopnykh-gazov-ford-trans...
9651,F22-474,Датчик давления наддува FORD 1.6 TDCI,NaN,"1256481, 1468879, 1231463, 1256481, MAZDA Y601...",Система топливная,Датчик давления воздуха (MAP / наддув / выхлоп),8e4bf915-a4bb-4f74-8365-0ca5bb3f8324,datchik-davleniia-nadduva-ford-1-6-tdci
9652,F22-300,Датчик давления фреона Mondeo 1993-2000 / Tran...,/upload/iblock/dd3/dlji06uet8z2kkjp92a1r03qjhy...,"95BW19E561AA, 3317869, 2S6H19E561AA, 1217951, ...",Электрооборудование,Датчики кондиционера,1bfc3273-7923-49b1-b15e-b1fe40bb3d1e,datchik-davleniia-freona-mondeo-1993-2000-tran...
9653,F22-473,Датчик давления фреона,NaN,"95BW 19E561 AA , 1016565",Электрооборудование,Датчики кондиционера,fed2b15f-56f9-4be8-a88d-be7e86496f9f,datchik-davleniia-freona


In [ ]:
# catalogue['name'].value_counts()

## Временное решение для дубликатов (пропустить потом)

In [81]:
# tmp_counts = catalogue["name"].value_counts().reset_index()

In [86]:
# tmp_counts = tmp_counts[(tmp_counts["count"] > 1) & (tmp_counts["name"] != "1")]

In [90]:
# duplicates_to_export = catalogue[catalogue["name"].isin(tmp_counts["name"])].copy()
#
# duplicates_to_export["link"] = (
#         "https://ford-parts.com.ru/catalog/"
#         + duplicates_to_export["bitrix_id"].astype(str)
#         + "/"
# )


In [92]:
# duplicates_to_export.to_excel("data/duplicates.xlsx", index=False)

## Создать категории и сабкатегории

In [212]:
category_images = pd.read_csv("data/images_categories_tcf.csv")

In [213]:
category = pd.DataFrame(catalogue["category"].drop_duplicates())
category = category.rename(columns={"category": "name"})
ids = [str(uuid4()) for i in range(len(category))]
category['category_id'] = ids
category = category.merge(category_images, on='name', how='left')
category

,name,category_id,slug,old_image
0,Кузов,f3eed464-3348-4942-af2b-448e2712fc49,kuzov-ford,/upload/iblock/eb0/a7whmp91qy522054431cyx4dcb1...
1,Двигатель,381dea74-66d4-4952-a5f2-2d9092ce54dd,dvigatel-ford,/upload/iblock/62b/iuf1syct13rhi50he2p4x3ns068...
2,Аксессуары,cbe00365-ea13-496e-ab58-dff7f20cd4f3,aksessuary-ford,/upload/iblock/4ac/eo5pnechgqjs39tiqk71bruzl5z...
3,Система выхлопная и Турбины,e36ee558-faee-40e0-8b23-f0764341fc59,sistema-vyhlopnaya-i-turbiny-ford,/upload/iblock/65b/xkh1awpw1dlkna93chdivy7pe05...
4,Подвеска,7e3f43ce-872c-4181-b4f6-4efc4bab1647,podveska-ford,/upload/iblock/ef3/lke05y6nond3gcnsle9xd2q4swb...
5,Система охлаждения,a1ab1e57-f0d0-423f-8a3c-d18ff727548d,sistema-ohlazhdeniya-ford,/upload/iblock/b36/3x283iz4lbl98hfjum2sr1fup97...
6,Рулевое управление,984cc38a-fc30-420b-9bca-bfc0f4d0069a,rulevoe-upravlenie-ford,/upload/iblock/af1/mlo1135rjsuqv24ijzv18801sns...
7,Система зажигания и управ-я,d4288f36-c424-4d11-9ebd-239b19b26861,sistema-zazhiganiya-i-uprav-ya-ford,/upload/iblock/da3/tkw3vi1jlgxdxrwb0p35l1ipbdl...
8,Стартеры и их компоненты,c690a217-8280-4782-bc75-0e7d83d048e9,startery-i-ih-komponenty-ford,/upload/iblock/deb/2jk27o1ymwwo1fc0tzllhi184ii...
9,Колеса,87ca9f28-1625-49d8-b5e9-344ac3e2ae7f,kolesa-ford,/upload/iblock/619/7s44rkdn1v0h0njquwe90pjyma8...


In [214]:
sub_category = pd.DataFrame(catalogue["sub_category"].drop_duplicates())
sub_category = sub_category.rename(columns={"sub_category": "name"})
ids = [str(uuid4()) for i in range(len(sub_category))]
sub_category['sub_category_id'] = ids
sub_category["slug"] = [slugify(n) for n in sub_category["name"].str.lower()]
sub_category= sub_category.merge(category_images[["name", "old_image"]], on='name', how='left')
sub_category

,name,sub_category_id,slug,old_image
0,Механизмы замков,5e196a7c-2e7d-429a-a5c5-fc687d00e4de,mekhanizmy-zamkov,NaN
1,Полукольца коленвала,4eacc7ed-925f-4513-a86c-17174a8000fd,polukoltsa-kolenvala,/upload/iblock/1bd/17v2do3px7367ok0ddt1qqlthzx...
2,Автомобильный набор,35747f59-2609-4a98-a4b6-7ccab888ec16,avtomobilnyi-nabor,NaN
3,Актуатор Турбины,8753c5dc-8c54-41de-8375-4b2d07790bca,aktuator-turbiny,NaN
4,Амортизатор,54ce96e4-4183-4038-a9bb-525987fabec9,amortizator,NaN
...,...,...,...,...
284,Маслянный радиаторы,4b94df06-1d93-4193-9bfc-a5c2536ff178,masliannyi-radiatory,/upload/iblock/e51/xk5g2odgxsgfmj4tgcwj1593otv...
285,Жидкость стеклоомывателя,8278e3df-77dc-4421-81ba-36d688875959,zhidkost-stekloomyvatelia,NaN
286,Батарейки,dc26127f-d768-494d-8fa4-9c0c39583276,batareiki,NaN
287,Датчик давления воздуха (MAP / наддув / выхлоп),13b50556-89ae-409e-8879-2616d407fb4f,datchik-davleniia-vozdukha-map-nadduv-vykhlop,NaN


In [217]:
pre_final = catalogue.merge(category, left_on="category", right_on="name", how="left").merge(sub_category, left_on="sub_category", right_on="name", how="left")

In [220]:
pre_final[pre_final["slug_x"] == "fiesta-2008"]
# pre_final["slug_x"].value_counts()

,bitrix_id,name_x,old_image_x,cross_number,category,sub_category,id,slug_x,name_y,category_id,slug_y,old_image_y,name,sub_category_id,slug,old_image
5186,F18-1849,Fiesta 2008->,NaN,"1802625, 1894332, 1550271",Система тормозная,Рабочие тормозные цилиндры,5231370b-d569-4d9a-92e4-159b8461e864,fiesta-2008,Система тормозная,4f7e3282-62be-4ca1-a9d9-86b7a0713641,sistema-tormoznaya-ford,/upload/iblock/68e/ug1toh7bqcxmvezcg82ozdy2s3z...,Рабочие тормозные цилиндры,ab9f0a2a-0362-4e65-99cb-434f0b10cc27,rabochie-tormoznye-tsilindry,NaN
5687,F13-263,Fiesta 2008-,NaN,8V51 3K644 BB / 1758288 - MGA 67103,Рулевое управление,Рулевые карданы,03a0507c-b609-4b82-a187-f830cb2ee9ca,fiesta-2008,Рулевое управление,984cc38a-fc30-420b-9bca-bfc0f4d0069a,rulevoe-upravlenie-ford,/upload/iblock/af1/mlo1135rjsuqv24ijzv18801sns...,Рулевые карданы,fa39220e-1143-4778-80eb-be19c5d21c53,rulevye-kardany,NaN
5764,F13-503,Fiesta 2008 ->,/upload/iblock/9e0/kjf3trvwgxawthik5xtkogzfpfv...,"8V51 3200 CJ, 1753627, 8V51 3200 CH - 77R078",Рулевое управление,Рулевые рейки,1e113a33-0060-4825-aa8b-08f58d725b41,fiesta-2008,Рулевое управление,984cc38a-fc30-420b-9bca-bfc0f4d0069a,rulevoe-upravlenie-ford,/upload/iblock/af1/mlo1135rjsuqv24ijzv18801sns...,Рулевые рейки,618173d2-6af3-4824-a9e6-aa9543371c09,rulevye-reiki,NaN
8166,F8-1883,"""Fiesta"" 2008-",/upload/iblock/c59/575t2p05kdiaovxajotr7vtv3p8...,"1580683, 8A61 A42528 AA",Кузов,Эмблемы и надписи,fc504b57-97b8-4570-84ac-631c82d8612b,fiesta-2008,Кузов,f3eed464-3348-4942-af2b-448e2712fc49,kuzov-ford,/upload/iblock/eb0/a7whmp91qy522054431cyx4dcb1...,Эмблемы и надписи,d9c29a22-fd0b-49ab-be31-48490934135c,emblemy-i-nadpisi,NaN


## Предложения (20 000 -> 25 000 за 4 месяца)
Скрестить с прайс-листом для цен и кол-ва

In [14]:
offers = pd.read_csv("data/offers.csv", sep=",")

In [15]:
offers[offers["IE_NAME"] == "SPRINTER 2006->"]

,IE_XML_ID,IE_NAME,IE_ID,IE_ACTIVE,IE_ACTIVE_FROM,IE_ACTIVE_TO,IE_PREVIEW_PICTURE,IE_PREVIEW_TEXT,IE_PREVIEW_TEXT_TYPE,IE_DETAIL_PICTURE,...,IE_SORT,IE_TAGS,IP_PROP538,IP_PROP539,IP_PROP661,IP_PROP536,IP_PROP660,IC_GROUP0,IC_GROUP1,IC_GROUP2
19818,M11-98,SPRINTER 2006->,271988,Y,NaN,NaN,NaN,9064600096,text,NaN,...,500,NaN,BSG60705002,NaN,BSG60705002,SPRINTER 2006-&gt; [271987],BSG,NaN,NaN,NaN
19819,M11-99,SPRINTER 2006->,271989,Y,NaN,NaN,/upload/iblock/4bc/hqvv5ko1eqkwdz7ueg5z5kft5zq...,NaN,text,NaN,...,500,NaN,3392501,NaN,3392501,SPRINTER 2006-&gt; [271987],LEMFORDER,NaN,NaN,NaN
19820,M11-100,SPRINTER 2006->,271990,Y,NaN,NaN,/upload/iblock/eb4/g8psn7dwh8cggl4z86u7fpxlfk6...,NaN,text,NaN,...,500,NaN,KRB104,NaN,KRB104,SPRINTER 2006-&gt; [271987],KORTEX,NaN,NaN,NaN
20219,M17-6,SPRINTER 2006->,272928,Y,NaN,NaN,NaN,NaN,text,NaN,...,500,NaN,1006209845,NaN,1006209845,SPRINTER 2006-&gt; [272927],WAI,NaN,NaN,NaN
20220,M17-7,SPRINTER 2006->,272929,Y,NaN,NaN,NaN,NaN,text,NaN,...,500,NaN,1696,NaN,1696,SPRINTER 2006-&gt; [272927],ZEN,NaN,NaN,NaN
23234,368771,SPRINTER 2006->,368771,Y,NaN,NaN,/upload/iblock/328/l6o1yjn000t056l0t4hzigom9vw...,NaN,text,NaN,...,500,NaN,1300494SX,NaN,1300494SX,SPRINTER 2006-&gt; [271987],STELLOX,NaN,NaN,NaN
23527,376492,SPRINTER 2006->,376492,Y,NaN,NaN,/upload/iblock/041/dd3rqbmwdn3prvy5umj8b8dbl8u...,NaN,text,NaN,...,500,NaN,103034,NaN,103034,SPRINTER 2006-&gt; [271987],Febi,NaN,NaN,NaN


In [16]:
offers = offers[
    [
        "IE_XML_ID",
        "IE_NAME",
        "IE_ID",
        "IE_PREVIEW_PICTURE",
        "IP_PROP661",
        "IP_PROP660",
    ]
]

In [17]:
offers = offers.rename(
    columns={
        "IE_XML_ID": "offer_bitrix_id",
        "IE_NAME": "name",
        "IE_ID": "IE_ID",
        "IE_PREVIEW_PICTURE": "old_image",
        "IP_PROP661": "manufacturer_number",
        "IP_PROP660": "brand",
    }
)

In [39]:
offers[offers["IE_ID"] == "F18-1777"]

,offer_bitrix_id,name,IE_ID,old_image,manufacturer_number,brand


In [19]:
price_list = pd.read_excel("data/price_opt.xlsx")
price_list['ID'] = price_list['ID'].astype(str)

In [20]:
x = price_list.merge(offers, left_on="ID", right_on="offer_bitrix_id", how="left")

In [24]:
x['Название'].value_counts()

Название
1                                                        27
NGK  BPR6EFS (конус) под ключ 16 мм (1.8-2.0 DURATEC)    22
Колодки тормозные пер. Transit 1991-2000 150-190 R15"    22
Помпа воды 2.5D / TD Transit 1986-2000                   21
Шаровая Transit 1991-2000                                19
                                                         ..
Transit 2014- FWD задний                                  1
Transit 2006- задний (650.5mm) / 4X4                      1
Transit 2003-2005 передний газ с планкой                  1
Transit 2000-2013 задний (550mm)                          1
Тормозная жидкость FERODO DOT4                            1
Name: count, Length: 10616, dtype: int64

In [50]:
price_list[price_list["ID"].isin(filtered_catalogue["bitrix_id"])].to_excel("data/duplicates.xlsx")

In [42]:
offers[offers["offer_bitrix_id"] == "F18-1777"]

,offer_bitrix_id,name,IE_ID,old_image,manufacturer_number,brand
7866,F18-1777,Mondeo 1996-2015 / S-MAX 2007- / Galaxy 1996-2...,250496,NaN,LX0327,DELPHI


In [69]:
grouped_df = catalogue.groupby('name')['sub_category'].nunique()

# Фильтрация групп, где количество уникальных подсистем > 1
duplicate_names = grouped_df[grouped_df > 1].index

# Выбор строк из исходного DataFrame, где "Название" входит в список дубликатов
result = catalogue[catalogue['name'].isin(duplicate_names)]

In [71]:
result[result["name"] != "1"]

,bitrix_id,name,IE_ID,old_image,cross_number,IE_CODE,category,sub_category
1714,F17-129,1.6 D 1984-1988,240862,/upload/iblock/8ea/bl1h1klqn0jylzczdulzj1qh1pu...,"A840X8591JA, 5023817, EPW65, A830X8591TA, EPW6...",f17-129,Система охлаждения,Водяная помпа / прокладки водяной помпы
1729,F17-181,2.0 CVH Split Port USA,240929,/upload/iblock/20c/z5kqaa1cs9ai9ywfck81vmrndb7...,1480869; 2026506; LF5215100A; YS4Z8501AA,f17-181,Система охлаждения,Водяная помпа / прокладки водяной помпы
2849,F16-35,2.0 CVH Split Port USA,244618,/upload/iblock/e34/vojugekh5tkd0ud5tvjzkn80drp...,YU2Z 12259 AA,f16-35,Система зажигания и управ-я,Кабели высокого напряжения
3507,F4-1153,1.0 Ecoboost,246920,/upload/iblock/b71/nyzjzrf5a1whsqzb5perm580oae...,"1771609 - 1771609, CM5G-6051-GC, 1939521, CM5Z...",f4-1153,Двигатель,Комплект прокладок
4850,F4-1823,ZETEC-E 1.6 / 1.8 / 2.0i 1993-1998 паразитный...,251121,/upload/iblock/628/4dqqd5dzgy1htvuk6xusugmer01...,"6635942, 928M 6M250 BC, F5RZ 6M25 0A , F-140222",f4-1823,Ремни и ролики,Натяжители ГРМ и ремней генератора
5088,F9-329,Escort 80-86 R,251773,NaN,NaN,f9-329,Оптика,Поворотники
5601,F4-2300,2.0 Ecoboost,253635,/upload/iblock/143/ufb9u5czo2epq6zwmajkbnmomtl...,AG9Z6108L,f4-2300,Двигатель,Поршень ДВС
5669,F4-2460,1.0 Ecoboost,253864,NaN,NaN,f4-2460,Двигатель,Прокладка под головку
5728,F4-2643,2.0 Ecoboost,254108,NaN,NaN,f4-2643,Двигатель,Прокладка под головку
5821,F15-179,1.6 D 1984-1988,254433,NaN,"84FM9448AB, 6121889",f15-179,Система выхлопная и Турбины,Прокладки выпускного тракта и турбин
